# Coordinate-Pair Eccentricity (CPE)


In [11]:
# Prereq Packages
import numpy as np
import csv
import MDAnalysis as mda
import dask
import dask.multiprocessing
from dask.distributed import Client, LocalCluster, performance_report


## Importing Data

GROMACS provides built-in analysis functions, which we have made use of to compute the semi-axes which this analysis is based on. Here's a short code to parse the .xvg output of GROMACS' "gyrate" analysis, which contains the semiaxes lengths we're interested in. "gmx gyrate" will also generate the radius of gyration, but we are interested in the ellipsoidal representation to measure shape, not a spherical model for looking at diffusion (so that's why a column is loaded in but not used). 

Code to compute the semi-axes directly from a molecular dynamics trajectory will be included later.

In [3]:
# Making a function to quickly read in xmgrace files
def xmgrace(io):
    '''
    Reads the xvg output from GMX gyrate.
    Returns time and semi-axes a, b, and c as a numpy array
    Column 0 - time
    (Column 1 - radius of gyration, not used)
    Column 2 - c (smallest)
    Column 3 - b
    Column 4 - a (largest)
    '''
    f = open(io,'r')
    lines = f.readlines()
    array = []
    for text in lines:
        if not any(value in text for value in ("#","@","&")):
            x = text.strip()
            x = x.split()
            columns = len(x)
            for i in range(columns):
                x[i] = float(x[i])
            array.append(x)
    rows = len(array)
    array2 = []
    for i in range(columns):
        y = np.zeros(rows)
        for j in range(rows):
            y[j] = array[j][i]
        array2.append(y)
    return np.transpose(np.asarray(array2))


## Computing CPE

It's really a simple function. The more difficult part is computing the semi-axis lengths, which is assumed already done at this point.

In [4]:
def CPE(a,b,c):
    '''
    Computing CPE from semi-axis lengths
    Returns 2 values, eab and eac, both on [0,1]
    
    a > b > c
    If you get errors, then your semi-axis order has been
    flipped somewhere.
    '''
    eab = np.sqrt(1-(b**2/a**2))
    eac = np.sqrt(1-(c**2/a**2))
    return eab,eac


## Computing the Semi-Axes

(displays fine as a jupyter notebook, less fine on GitHub. Sorry)

Here are a few variations for computing the semi-axes directly from a molecular dynamics trajectory. The package relied upon for manipulating the trajectory, MDAnalysis, has a function for computing the moments of inertia directly. If you are working with something other than a molecular dynamics trajectory, you will need to find an alternate method of computing the moments of inertia, but all that will really do is change the input of the code below. 

The semi-axes for the ellipse are being solved for with linear algebra. The moments of inertia for an ellipsoid can be given by the equations:

$$
A = \frac{M}{5}(b^2 + c^2) \\
B = \frac{M}{5}(a^2 + c^2) \\
C = \frac{M}{5}(a^2 + b^2)
$$

where $A$, $B$, and $C$ are the principal moments of inertia, $M$ is the mass, and $a$, $b$, and $c$ are the semi-axes. To keep things straight, keep in mind that we are naming the semi-axes according to the convention $a \geq b \geq c$. This also implies that $A\leq B \leq C$, so the order is flipped. 

We can recast those equations in matrix form to solve the system of equations:

$$
\begin{pmatrix} A \\ B \\ C \end{pmatrix} = \frac{M}{5}\begin{pmatrix} 0 & 1 & 1 \\ 1 & 0 & 1 \\ 1 & 1 & 0 \end{pmatrix} \begin{pmatrix}a^2 \\ b^2 \\ c^2 \end{pmatrix}
$$

$$
\frac{5}{M}\begin{pmatrix} 0 & 1 & 1 \\ 1 & 0 & 1 \\ 1 & 1 & 0 \end{pmatrix}^{-1}\begin{pmatrix} A \\ B \\ C \end{pmatrix} = \begin{pmatrix} 0 & 1 & 1 \\ 1 & 0 & 1 \\ 1 & 1 & 0 \end{pmatrix}^{-1}\begin{pmatrix} 0 & 1 & 1 \\ 1 & 0 & 1 \\ 1 & 1 & 0 \end{pmatrix} \begin{pmatrix}a^2 \\ b^2 \\ c^2 \end{pmatrix} \\
= \begin{pmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{pmatrix}\begin{pmatrix}a^2 \\ b^2 \\ c^2 \end{pmatrix} = \begin{pmatrix}a^2 \\ b^2 \\ c^2 \end{pmatrix}
$$

Where:

$$
\begin{pmatrix} 0 & 1 & 1 \\ 1 & 0 & 1 \\ 1 & 1 & 0 \end{pmatrix}^{-1} = \begin{pmatrix}-1/2 & 1/2 & 1/2 \\ 1/2 & -1/2 & 1/2 \\ 1/2 & 1/2 & -1/2 \end{pmatrix}
$$

This is the basis of the manipulations done in the code below.

### IMPORTANT NOTE ABOUT MULTIPROCESSING:

MDAnalysis is recently released a major update, v2.0. This new update can handle multiprocessing, but older versions cannot. If you would like to use multiprocessing, make sure you have updated to the latest version (>v2.0). 

In [5]:
'''
Semi-Axes Finder V1:
Computing for a single frame of a trajectory

Note: I made use of an atom selection string rather than the actual
    AtomGroup object for whatever reason. It should work fine either way,
    whichever is more convenient for you.
'''
def Semiaxes_Single(frame,Universe,AtomSelect):
    '''
    Computes the semi-axes for a given selection of atoms in MD traj.
    
    frame - an integer number for the frame of the trajectory you'd like
            to analyze
    Universe - the general object MDAnalysis creates containing the trajectory
    AtomSelect - a string with the selection language for the atoms of interest
                See the link for more information on MDAnalysis selection language:
                https://userguide.mdanalysis.org/stable/selections.html
    
    Returns the semi-axis lengths as a numpy array from smallest to largest (c,b,a)
    
    Lengths have same units as MDAnalysis, default of Angstroms unless
    you specify otherwise
    '''
    # creating AtomGroup object
    selection = Universe.select_atoms(AtomSelect)
    # changing active frame of the universe to desired frame
    Universe.trajectory[frame]
    
    # computes the moments of inertia and their vector directions
    # moments_val for the MoI themselves, princ_vec for the vector directions
    moments_val,princ_vec = np.linalg.eig(selection.moment_of_inertia())
    # getting mass of the selected group of atoms
    Micelle_Mass = selection.total_mass()
    # sortinng MoI and vectors by size of MoI
    idx = moments_val.argsort()[::-1]
    moments_val = moments_val[idx]
    princ_vec = princ_vec[:,idx]
    # Array to solve for axis lengths 
    Inverter = np.array([[-1,1,1],[1,-1,1],[1,1,-1]])*0.5
    # converting MoI to axis length eigenvalues
    semiaxis = np.sqrt((5/Micelle_Mass)*(np.matmul(Inverter,np.transpose(moments_val))))
    
    # UNITS IN ANGSTROMS
    return semiaxis


In [14]:
'''
Semi-Axes Finder V2:
Computing for all frames of a trajectory, serial

Note: I made use of an atom selection string rather than the actual
    AtomGroup object for whatever reason. It should work fine either way,
    whichever is more convenient for you.
'''
def Semiaxes_All(Universe,AtomSelect):
    '''
    Computes the semi-axes for all frames in a trajectory
    
    Universe - the general object MDAnalysis creates containing the trajectory
    AtomSelect - a string with the selection language for the atoms of interest
                See the link for more information on MDAnalysis selection language:
                https://userguide.mdanalysis.org/stable/selections.html
    
    Returns the semi-axis lengths as a 2d numpy array
    axis 0 (rows) are time, columns for each semi-axis,
    from smallest to largest (time,c,b,a)
    
    Lengths have same units as MDAnalysis, default of Angstroms unless
    you specify otherwise
    '''
    # creating AtomGroup object
    selection = Universe.select_atoms(AtomSelect)
    
    # prepping output
    semiaxis = np.empty((Universe.trajectory.n_frames,4))
    
    # cycling through trajectory
    for i in range(Universe.trajectory.n_frames):
        # computes the moments of inertia and their vector directions
        # moments_val for the MoI themselves, princ_vec for the vector directions
        moments_val,princ_vec = np.linalg.eig(selection.moment_of_inertia())
        # getting mass of the selected group of atoms
        Micelle_Mass = selection.total_mass()
        # sortinng MoI and vectors by size of MoI
        idx = moments_val.argsort()[::-1]
        moments_val = moments_val[idx]
        princ_vec = princ_vec[:,idx]
        # Array to solve for axis lengths 
        Inverter = np.array([[-1,1,1],[1,-1,1],[1,1,-1]])*0.5
        # converting MoI to axis length eigenvalues
        semiaxis_temp = np.sqrt((5/Micelle_Mass)*(np.matmul(Inverter,np.transpose(moments_val))))
        # getting time
        semiaxis[i,0] = Universe.trajectory.time
        # putting semiaxes in final output
        semiaxis[i,1:] = semiaxis_temp
    

    # UNITS IN ANGSTROMS
    return semiaxis

In [17]:
'''
Semi-Axes Finder V3:
Computing for all frames of a trajectory, parallel

Note: I made use of an atom selection string rather than the actual
    AtomGroup object for whatever reason. It should work fine either way,
    whichever is more convenient for you.
Note 2: This multiprocessing code assumes you are running on some sort
    of cluster. I'm not an expert on Dask, but I *think* it should work
    on a standard desktop computer, too, so it's a more general option. 
    If that's not the case, you'll have to dig into Dask documentation
    yourself.
'''
def Semiaxes_All(Universe,AtomSelect):
    '''
    Computes the semi-axes for all frames in a trajectory in parallel
    
    Universe - the general object MDAnalysis creates containing the trajectory
    AtomSelect - a string with the selection language for the atoms of interest
                See the link for more information on MDAnalysis selection language:
                https://userguide.mdanalysis.org/stable/selections.html
    
    Returns the semi-axis lengths as a 2d numpy array
    axis 0 (rows) are time, columns for each semi-axis,
    from smallest to largest (time,c,b,a)
    
    Lengths have same units as MDAnalysis, default of Angstroms unless
    you specify otherwise
    '''   
    # Multiprocessing requires that we reorganize some things
    def Semiaxes_Single(frame,Universe,AtomSelect):
        
        # changing active frame of the universe to desired frame
        Universe.trajectory[frame]

        # computes the moments of inertia and their vector directions
        # moments_val for the MoI themselves, princ_vec for the vector directions
        moments_val,princ_vec = np.linalg.eig(selection.moment_of_inertia())
        # getting mass of the selected group of atoms
        Micelle_Mass = selection.total_mass()
        # sortinng MoI and vectors by size of MoI
        idx = moments_val.argsort()[::-1]
        moments_val = moments_val[idx]
        princ_vec = princ_vec[:,idx]
        # Array to solve for axis lengths 
        Inverter = np.array([[-1,1,1],[1,-1,1],[1,1,-1]])*0.5
        # converting MoI to axis length eigenvalues
        semiaxis = np.sqrt((5/Micelle_Mass)*(np.matmul(Inverter,np.transpose(moments_val))))
        # getting time
        time = Universe.trajectory.time
        # getting unified output
        output = np.concatenate((np.array([time]),semiaxis))
        # UNITS IN ANGSTROMS
        return output
    
    # Function to compute only a section of the trajectory
    # It will just cycle through a start/stop and call the single frame
    # to compute
    def Batch_Comp(myrange,Universe,AtomSelect):
        # prepping output
        partial_results = np.empty((len(range),4))
        # i will not be the same as the output index, need separate counter
        call = 0
        # cycling through this batches frames
        for i in myrange:
            partial_results[call,:] = Semiaxes_Single(i,Universe,AtomSelect)
            call += 1
        return partial_results
    
    # creating AtomGroup object, lets only do this a single time
    selection = Universe.select_atoms(AtomSelect)
    
    # multiprocessing, not threading
    dask.config.set(scheduler='processes')
    
    # Setting the main multiprocessing
    if __name__ == '__main__':
        # getting local architecture for Dask
        client = Client()
        # getting number of cores available
        partitions = len(client.ncores().values())
        # splitting trajectory into batches for processing
        batch_size = Universe.trajectory.n_frames//partitions
        # creating start/stop for batch ranges
        start = np.empty(partitions)
        stop = np.empty(partitions)
        for i in range(partitions):
            start[i] = i*batch_size
            stop[i] = (i+1)*batch_size
        # adjusting last batch to make sure we get all frames (accounts for remainders)
        stop[-1] = Universe.trajectory.n_frames
        
        # starting multiprocessing
        # first we make a list of dask.delayed jobs (lazy functions that are actually evaluated later)
        job_list = []
        for i in range(partitions):
            job_list.append(dask.delayed(Batch_Comp)(range(int(start[i]),int(stop[i])),
                                                    Universe,selection))
        
        # creating futures because... it's how Dask works
        futures = client.compute(job_list)
        
        # This step actually starts computation of the functions
        result = client.gather(futures)
        
        # code is blocked here (won't evaluate further until computation finishes)
        # once done, we collect the results into a single output
        # prepping output
        semiaxis = np.empty((0,4))
        # cycling through each partitions results
        for i in range(len(result)):
            semiaxis = np.concatenate((semiaxis,results[i][0]),axis=0)

    # UNITS IN ANGSTROMS
    return semiaxis


## Writing Output

If you are not using GROMACS and need to compute CPE using the code above, you will also need to write the output. 

Here is an easy outline you can use based on the writing function I used for convexity (simply adding 1 value for 2 CPE values, or 2 extra values for all 3 semi-axis lengths).

In [ ]:
# a writer for the CPE values
def CPEWriter(file,array):
    '''
    Writes your results to a csv file
    
    Assumes that you have an N x 3 array with time, eab, eac in each column
    N is the number of frames computed
    
    File is your designated path and filename
    Array is the N x 3 array containing your results
    '''
    with open(file,'w') as fh:
        for i in range(array.shape[0]):
            fh.write(str(array[i,0])+','+str(array[i,1])+','+str(array[i,2])+'\n')

# a writer for the semiaxes values
def SemiAxesWriter(file,array):
    '''
    Writes your results to a csv file
    
    Assumes that you have an N x 4 array with time,a,b,c in each column
    N is the number of frames computed
    
    File is your designated path and filename
    Array is the N x 4 array containing your results
    '''
    with open(file,'w') as fh:
        for i in range(array.shape[0]):
            fh.write(str(array[i,0])+','+str(array[i,1])+','+str(array[i,2])+','+str(array[i,3])+'\n')


## Reading Input

If you wrote the results to file using the above code, then you'll need to read it, too. Here's some code to read those files back in.

In [ ]:
# a reader for the CPE values
def CPE_reader(IO):
    '''
    Reads your results from a csv file created using CPEWriter
    
    IO is the path and filename for your csv file
    
    Outputs an N x 3 array with time, eab, eac in each column
    N is the number of frames computed
    '''
    output = np.empty((0,2))
    with open(IO,newline='') as csvfile:
        file = csv.reader(csvfile,delimiter=',')
        for row in file:
            output = np.append(output,np.array([[row[0],row[1],row[2]]]),axis=0)
    return output.astype(np.float64)

# a reader for the semiaxes values
def SemiAxes_reader(IO):
    '''
    Reads your results from a csv file created using SemiAxesWriter
    
    IO is the path and filename for your csv file
    
    Outputs an N x 4 array with time,a,b,c in each column
    N is the number of frames computed
    '''
    output = np.empty((0,2))
    with open(IO,newline='') as csvfile:
        file = csv.reader(csvfile,delimiter=',')
        for row in file:
            output = np.append(output,np.array([[row[0],row[1],row[2],row[3]]]),axis=0)
    return output.astype(np.float64)
